In [42]:
from pymongo import MongoClient
import pandas as pd
import json as js
import folium

In [43]:
# It is a problem to have many companies with multiple office locations. I (we) solved that
# using "unwind", and "match" to link the whole query

def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

db, coll = connectCollection('companies','companies')
pipeline = [
    { "$unwind": "$offices"},
    {"$match":{ "$and": [ {"number_of_employees":{"$gt":100} }, {"founded_year":{"$gt":2000}},{"funding_rounds.raised_amount":{"$gt":1000000}}] }}
    
 ]
results = list(coll.aggregate(pipeline))

In [102]:
location_raw=[]
name=[]
lon=[]
lat=[]

for element in results:
    for coord in element["offices"].items():
        name.append(element["name"])
        #print(coord)
        
        if coord[0]=="longitude":
            lon.append(coord[1])
        elif coord[0]=="latitude":
            lat.append(coord[1])

for x in zip(lon,lat):
    location_raw.append(list(x))

In [103]:
print((location_raw))

[[-122.151801, 37.41605], [-6.267494, 53.344104], [-73.9792469, 40.7557162], [-122.4169244, 37.7768052], [-77.245195, 38.926172], [-73.996876, 40.724604], [-118.380954, 34.0263021], [None, None], [None, None], [-83.167884, 42.557958], [-118.1327468, 34.1494713], [None, None], [None, None], [None, None], [None, None], [-111.938739, 33.429859], [-122.0899512, 37.4233896], [-73.995625, 40.726155], [-122.07948, 37.392936], [None, None], [None, None], [None, None], [-122.402195, 37.786183], [None, None], [None, None], [None, None], [None, None], [-122.206893, 37.484619], [-122.39071, 37.779507], [-122.079017, 37.393679], [None, None], [None, None], [None, None], [None, None], [-122.073635, 37.423258], [-122.143701, 37.425801], [-122.325248, 37.561877], [-122.178557, 37.479952], [-80.004195, 40.437985], [-121.8731805, 37.6637279], [-77.3592746, 38.9590083], [-0.1394759, 51.5170385], [-97.717667, 30.407545], [-122.4186, 37.7753], [-121.935688, 37.288974], [-71.086596, 42.362993], [None, None]

In [54]:
    for y in x["offices"]:
        print(y)
        #print(y["longitude"])
        '''
        longitude=y["longitude"]
        latitude=y["latitude"]
        loc = {
            'type':'Point',
            'coordinates':[longitude, latitude]
        }
        location_list.append(loc)'''

TypeError: string indices must be integers

In [34]:
        location_raw.append([coords["longitude"],coords["latitude"]])
#print(location_raw)'''

TypeError: string indices must be integers

In [47]:
print(len(name),len(location_list))

In [50]:
l={"name":name, "coords":location_raw}
data_locations=pd.DataFrame(data=l)

In [51]:
data_locations

,name,coords
0,Facebook,"[None, None]"
1,Facebook,"[None, None]"
2,Facebook,"[None, None]"
3,Twitter,"[None, None]"
4,AddThis,"[None, None]"
...,...,...
535,FleetMatics,"[None, None]"
536,FleetMatics,"[None, None]"
537,800APP,"[None, None]"
538,InfoLogix,"[None, None]"


In [55]:
world_map=folium.Map(location=[52.5200,13.4050], zoom_start=12)
for place in location_raw["coords"]:
    print(place)
    

TypeError: list indices must be integers or slices, not str

In [34]:
l={"name":name, "location":location_list}
data_locations=pd.DataFrame(data=l)

In [35]:
data_locations

,name,location
0,Facebook,"{'type': 'Point', 'coordinates': [-122.151801,..."
1,Facebook,"{'type': 'Point', 'coordinates': [-6.267494, 5..."
2,Facebook,"{'type': 'Point', 'coordinates': [-73.9792469,..."
3,Twitter,"{'type': 'Point', 'coordinates': [-122.4169244..."
4,AddThis,"{'type': 'Point', 'coordinates': [-77.245195, ..."
...,...,...
535,FleetMatics,"{'type': 'Point', 'coordinates': [None, None]}"
536,FleetMatics,"{'type': 'Point', 'coordinates': [None, None]}"
537,800APP,"{'type': 'Point', 'coordinates': [None, None]}"
538,InfoLogix,"{'type': 'Point', 'coordinates': [-75.1069232,..."


In [43]:
world_map=folium.Map(location=[52.5200,13.4050], zoom_start=12)
for company in data_locations:
world_map

In [50]:
#si quiero añadir información a la colección $set (para el geoindex)
for place in rest:
    value = {"$set": {'location':getLocation(place)}}
    coll.update_one(place,value)

In [51]:
# eliminar las antiguas coordenadas
for place in rest:
    value = {"$unset": {'offices.latitude':''}}
    value = {"$unset": {'offices.longitude':''}}
    coll.update_one(place,value)
    # <collection>.update_one(<document>,<value>)

In [54]:
'''
prefered_city = coll.find(
    {"location":
     {"$near":
      {"$geometry":
       {"type":"Point",
        "coordinates":[-73.9712,40.7831]
       },
       "$maxDistance":10000
      }
     }
    }
)
prefered_city=list(prefered_city)
'''

'\nprefered_city = coll.find(\n    {"location":\n     {"$near":\n      {"$geometry":\n       {"type":"Point",\n        "coordinates":[-73.9712,40.7831]\n       },\n       "$maxDistance":10000\n      }\n     }\n    }\n)\nprefered_city=list(prefered_city)\n'

In [58]:
map_city=folium.Map(location=[40.7221,-73.9712], zoom_start=12)
map_city
'''
for wendy in manhattanWendys:
    folium.Marker(wendy['location']['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='red'), 
                   ).add_to(map_city)
    
for wendy in manhattanWendys[:5]:
    folium.Marker(wendy['location']['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='blue'), 
                   ).add_to(map_city)
'''

"\nfor wendy in manhattanWendys:\n    folium.Marker(wendy['location']['coordinates'][::-1],\n                    radius=2,\n                    icon=folium.Icon(icon='cloud',color='red'), \n                   ).add_to(map_city)\n    \nfor wendy in manhattanWendys[:5]:\n    folium.Marker(wendy['location']['coordinates'][::-1],\n                    radius=2,\n                    icon=folium.Icon(icon='cloud',color='blue'), \n                   ).add_to(map_city)\n"